<img src="https://bit.ly/2VnXWr2" width="100" align="left">

# Project | API and Web Data Scraping

## Import Useful Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import re
import math
import json
import requests
from pandas import json_normalize
from bs4 import BeautifulSoup

## Working On The API

### Getting the needed information from the API

In [2]:
url = "https://pikalytics.com/api/p/2020-01/ss-1760"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"}

response = requests.get(url, headers = headers) 
results = response.json()
flattened_data = json_normalize(results)
pikalyticsDF = flattened_data

TypeError: 'bool' object is not iterable

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

### Drop the columns I don't need and clean the remaining information

In [3]:
pikalyticsDF = pikalyticsDF.drop(['abilities', 'raw_count', 'items', 'spreads', 'moves', 'stats.hp',
       'stats.atk', 'stats.def', 'stats.spa', 'stats.spd', 'stats.spe', 'ss'], axis=1)

NameError: name 'pikalyticsDF' is not defined

### As I had issues to properly cleaning the column team I will make a copy from it and treat it individually.

In [5]:
teams = pikalyticsDF["team"].tolist()

In [6]:
teamsDF = pd.DataFrame(teams)

In [7]:
teamsDF = teamsDF.drop([3,4,5,6,7,8,9,10,11], axis=1)

In [8]:
teamsDF = teamsDF.rename(columns={0:"Most common teammate",
                            1:"2nd most common teammate",
                            2:"3rd most common teammate"})
teamsDF.head()

,Most common teammate,2nd most common teammate,3rd most common teammate
0,"{'pokemon': 'Excadrill', 'percent': '38.536', 'types': ['ground', 'steel']}","{'pokemon': 'Arcanine', 'percent': '35.593', 'types': ['fire']}","{'pokemon': 'Dragapult', 'percent': '33.117', 'types': ['dragon', 'ghost']}"
1,"{'pokemon': 'Togekiss', 'percent': '45.687', 'types': ['fairy', 'flying']}","{'pokemon': 'Dragapult', 'percent': '30.663', 'types': ['dragon', 'ghost']}","{'pokemon': 'Tyranitar', 'percent': '28.847', 'types': ['rock', 'dark']}"
2,"{'pokemon': 'Togekiss', 'percent': '48.228', 'types': ['fairy', 'flying']}","{'pokemon': 'Excadrill', 'percent': '38.133', 'types': ['ground', 'steel']}","{'pokemon': 'Arcanine', 'percent': '34.788', 'types': ['fire']}"
3,"{'pokemon': 'Togekiss', 'percent': '59.572', 'types': ['fairy', 'flying']}","{'pokemon': 'Tyranitar', 'percent': '47.122', 'types': ['rock', 'dark']}","{'pokemon': 'Dragapult', 'percent': '40.479', 'types': ['dragon', 'ghost']}"
4,"{'pokemon': 'Duraludon', 'percent': '36.074', 'types': ['steel', 'dragon']}","{'pokemon': 'Togekiss', 'percent': '32.431', 'types': ['fairy', 'flying']}","{'pokemon': 'Arcanine', 'percent': '25.983', 'types': ['fire']}"


In [9]:
teamsDF_First = teamsDF["Most common teammate"].tolist()
teamsDF_First = pd.DataFrame(teamsDF_First)
teamsDF_First["Most common teammate"] = teamsDF_First["pokemon"].astype(str) + " " + teamsDF_First["percent"].astype(str) +"% " + teamsDF_First["types"].astype(str)
teamsDF_First = teamsDF_First.drop(["pokemon","percent","types"], axis=1)

In [10]:
teamsDF_Second = teamsDF["2nd most common teammate"].tolist()
teamsDF_Second = pd.DataFrame(teamsDF_Second)
teamsDF_Second["2nd most common teammate"] = teamsDF_Second["pokemon"].astype(str) + " " + teamsDF_Second["percent"].astype(str) +"% " + teamsDF_Second["types"].astype(str)
teamsDF_Second = teamsDF_Second.drop(["pokemon","percent","types"], axis=1)

In [11]:
teamsDF_Third = teamsDF["3rd most common teammate"].tolist()
teamsDF_Third = pd.DataFrame(teamsDF_Third)
teamsDF_Third["3rd most common teammate"] = teamsDF_Third["pokemon"].astype(str) + " " + teamsDF_Third["percent"].astype(str) +"% " + teamsDF_Third["types"].astype(str)
teamsDF_Third = teamsDF_Third.drop(["pokemon","percent","types"], axis=1)

### Now I'll create new columns in the original dataset to add the information I wanted from the DF I cleaned separately

In [12]:
pikalyticsDF = pikalyticsDF.drop(["team"], axis=1)

In [13]:
pikalyticsDF["Most common teammates"]= teamsDF_First["Most common teammate"]
pikalyticsDF["2nd most common teammates"]= teamsDF_Second["2nd most common teammate"]
pikalyticsDF["3rd most common teammates"]= teamsDF_Third["3rd most common teammate"]

In [14]:
pikalyticsDF.head(5)

,name,types,percent,ranking,viability,Most common teammates,2nd most common teammates,3rd most common teammates
0,Togekiss,"[fairy, flying]",46.0,1,B,NaN,NaN,NaN
1,Arcanine,[fire],36.0,2,B,NaN,NaN,NaN
2,Dragapult,"[dragon, ghost]",31.0,3,B,NaN,NaN,NaN
3,Excadrill,"[ground, steel]",30.0,4,B,NaN,NaN,NaN
4,Whimsicott,"[grass, fairy]",29.0,5,B-,NaN,NaN,NaN


In [15]:
pikalyticsDF.loc[pikalyticsDF["Most common teammates"].isnull(), "Most common teammates"] = teamsDF_First["Most common teammate"]
pikalyticsDF.loc[pikalyticsDF["2nd most common teammates"].isnull(), "2nd most common teammates"] = teamsDF_Second["2nd most common teammate"]
pikalyticsDF.loc[pikalyticsDF["3rd most common teammates"].isnull(), "3rd most common teammates"] = teamsDF_Third["3rd most common teammate"]

### Here it is the clean pikalytics dataframe

In [16]:
pikalyticsDF.head()

,name,types,percent,ranking,viability,Most common teammates,2nd most common teammates,3rd most common teammates
0,Togekiss,"[fairy, flying]",46.0,1,B,"Excadrill 38.536% ['ground', 'steel']",Arcanine 35.593% ['fire'],"Dragapult 33.117% ['dragon', 'ghost']"
1,Arcanine,[fire],36.0,2,B,"Togekiss 45.687% ['fairy', 'flying']","Dragapult 30.663% ['dragon', 'ghost']","Tyranitar 28.847% ['rock', 'dark']"
2,Dragapult,"[dragon, ghost]",31.0,3,B,"Togekiss 48.228% ['fairy', 'flying']","Excadrill 38.133% ['ground', 'steel']",Arcanine 34.788% ['fire']
3,Excadrill,"[ground, steel]",30.0,4,B,"Togekiss 59.572% ['fairy', 'flying']","Tyranitar 47.122% ['rock', 'dark']","Dragapult 40.479% ['dragon', 'ghost']"
4,Whimsicott,"[grass, fairy]",29.0,5,B-,"Duraludon 36.074% ['steel', 'dragon']","Togekiss 32.431% ['fairy', 'flying']",Arcanine 25.983% ['fire']


## Working On The Web

### Getting the needed information from the web

In [17]:
url1 = "https://www.wikidex.net/wiki/Lista_de_Pok%C3%A9mon_con_sus_estad%C3%ADsticas_base"

In [18]:
html = requests.get(url1).content
soup = BeautifulSoup(html, "lxml")

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>Lista de Pokémon con sus estadísticas base - WikiDex, la enciclopedia Pokémon</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Lista_de_Pokémon_con_sus_estadísticas_base","wgTitle":"Lista de Pokémon con sus estadísticas base","wgCurRevisionId":2388989,"wgRevisionId":2388989,"wgArticleId":12515,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Listas de Pokémon"],"wgBreakFrames":true,"wgPageContentLanguage":"es","wgPageContentModel":"wikitext","wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["

### Cleaning the data we got and generating a dataframe

In [19]:
table = soup.find_all("table", {"class":"tabpokemon sortable mergetable tablemanager"})[0]

In [20]:
rows = table.find_all("tr")
rows = [row.text.strip().split("\n") for row in rows]
rows = [list(filter(None,row)) for row in rows]
wikidexDF = pd.DataFrame(rows)

In [21]:
wikidexDF.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,#,Pokémon,Mini sprite,PS,Ataque,Defensa,Ataque esp.,Defensa esp.,Velocidad,Promedio,σ,Suma
1,001,Bulbasaur,45,49,49,65,65,45,"53,00","8,64",318,None
2,002,Ivysaur,60,62,63,80,80,60,"67,50","8,90",405,None
3,003,Venusaur,80,82,83,100,100,80,"87,50","8,90",525,None
4,003,Mega-Venusaur,80,100,123,122,120,80,"104,17","18,75",625,None


In [22]:
wikidexDF = wikidexDF.drop([11], axis=1)

In [23]:
wikidexDF = wikidexDF.drop([0], axis=0)
wikidexDF.head()

,0,1,2,3,4,5,6,7,8,9,10
1,001,Bulbasaur,45,49,49,65,65,45,"53,00","8,64",318
2,002,Ivysaur,60,62,63,80,80,60,"67,50","8,90",405
3,003,Venusaur,80,82,83,100,100,80,"87,50","8,90",525
4,003,Mega-Venusaur,80,100,123,122,120,80,"104,17","18,75",625
5,004,Charmander,39,52,43,60,50,65,"51,50","9,00",309


In [24]:
colnames = ["National Dex #", "Pokémon", "HP", "Attack", "Defense", "Special Attack", "Special Defense", "Speed", "Mean", "SD", "Total"]

wikidexDF = pd.DataFrame(rows[1:], columns = colnames)

### Here it is the clean wikidex dataframe

In [25]:
wikidexDF.head()

,National Dex #,Pokémon,HP,Attack,Defense,Special Attack,Special Defense,Speed,Mean,SD,Total
0,001,Bulbasaur,45,49,49,65,65,45,"53,00","8,64",318
1,002,Ivysaur,60,62,63,80,80,60,"67,50","8,90",405
2,003,Venusaur,80,82,83,100,100,80,"87,50","8,90",525
3,003,Mega-Venusaur,80,100,123,122,120,80,"104,17","18,75",625
4,004,Charmander,39,52,43,60,50,65,"51,50","9,00",309


## Results

In [26]:
pikalyticsDF.to_csv("/Users/Miguel/Documents/GitHub/Ironhack exercises/Modulo 1/project-web/Your-code/Output/pikalyticsDF_clean.csv") 
wikidexDF.to_csv("/Users/Miguel/Documents/GitHub/Ironhack exercises/Modulo 1/project-web/Your-code/Output/wikidexDF_clean.csv")

# Bonus

## Combining Information

### Merging the resulting dataframes

In [27]:
Combined_DF = pd.merge(pikalyticsDF, wikidexDF, left_on="name", right_on="Pokémon")

In [28]:
Combined_DF.head()

,name,types,percent,ranking,viability,Most common teammates,2nd most common teammates,3rd most common teammates,National Dex #,Pokémon,HP,Attack,Defense,Special Attack,Special Defense,Speed,Mean,SD,Total
0,Togekiss,"[fairy, flying]",46.0,1,B,"Excadrill 38.536% ['ground', 'steel']",Arcanine 35.593% ['fire'],"Dragapult 33.117% ['dragon', 'ghost']",468,Togekiss,85,50,95,120,115,80,"90,83","23,35",545
1,Arcanine,[fire],36.0,2,B,"Togekiss 45.687% ['fairy', 'flying']","Dragapult 30.663% ['dragon', 'ghost']","Tyranitar 28.847% ['rock', 'dark']",059,Arcanine,90,110,80,100,80,95,"92,50","10,70",555
2,Dragapult,"[dragon, ghost]",31.0,3,B,"Togekiss 48.228% ['fairy', 'flying']","Excadrill 38.133% ['ground', 'steel']",Arcanine 34.788% ['fire'],887,Dragapult,88,120,75,100,75,142,"100,00","24,35",600
3,Excadrill,"[ground, steel]",30.0,4,B,"Togekiss 59.572% ['fairy', 'flying']","Tyranitar 47.122% ['rock', 'dark']","Dragapult 40.479% ['dragon', 'ghost']",530,Excadrill,110,135,60,50,65,88,"84,67","29,95",508
4,Whimsicott,"[grass, fairy]",29.0,5,B-,"Duraludon 36.074% ['steel', 'dragon']","Togekiss 32.431% ['fairy', 'flying']",Arcanine 25.983% ['fire'],547,Whimsicott,60,67,85,77,75,116,"80,00","17,91",480


### Drop columns that add no information

In [29]:
Combined_DF.columns

Index(['name', 'types', 'percent', 'ranking', 'viability',
       'Most common teammates', '2nd most common teammates',
       '3rd most common teammates', 'National Dex #', 'Pokémon', 'HP',
       'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed',
       'Mean', 'SD', 'Total'],
      dtype='object')

In [30]:
Combined_DF = Combined_DF.drop(['name', 'Total'], axis=1)
Combined_DF.head()

,types,percent,ranking,viability,Most common teammates,2nd most common teammates,3rd most common teammates,National Dex #,Pokémon,HP,Attack,Defense,Special Attack,Special Defense,Speed,Mean,SD
0,"[fairy, flying]",46.0,1,B,"Excadrill 38.536% ['ground', 'steel']",Arcanine 35.593% ['fire'],"Dragapult 33.117% ['dragon', 'ghost']",468,Togekiss,85,50,95,120,115,80,"90,83","23,35"
1,[fire],36.0,2,B,"Togekiss 45.687% ['fairy', 'flying']","Dragapult 30.663% ['dragon', 'ghost']","Tyranitar 28.847% ['rock', 'dark']",059,Arcanine,90,110,80,100,80,95,"92,50","10,70"
2,"[dragon, ghost]",31.0,3,B,"Togekiss 48.228% ['fairy', 'flying']","Excadrill 38.133% ['ground', 'steel']",Arcanine 34.788% ['fire'],887,Dragapult,88,120,75,100,75,142,"100,00","24,35"
3,"[ground, steel]",30.0,4,B,"Togekiss 59.572% ['fairy', 'flying']","Tyranitar 47.122% ['rock', 'dark']","Dragapult 40.479% ['dragon', 'ghost']",530,Excadrill,110,135,60,50,65,88,"84,67","29,95"
4,"[grass, fairy]",29.0,5,B-,"Duraludon 36.074% ['steel', 'dragon']","Togekiss 32.431% ['fairy', 'flying']",Arcanine 25.983% ['fire'],547,Whimsicott,60,67,85,77,75,116,"80,00","17,91"


### Reorder columns

In [31]:
column_order = ['National Dex #', 'Pokémon', 'types', 'HP', 'Attack', 'Defense',
       'Special Attack', 'Special Defense', 'Speed', 'Mean', 'SD', 'ranking', 'percent', 'viability', 'Most common teammates',
       '2nd most common teammates', '3rd most common teammates']
Combined_DF = Combined_DF[column_order]
Combined_DF.head()

,National Dex #,Pokémon,types,HP,Attack,Defense,Special Attack,Special Defense,Speed,Mean,SD,ranking,percent,viability,Most common teammates,2nd most common teammates,3rd most common teammates
0,468,Togekiss,"[fairy, flying]",85,50,95,120,115,80,"90,83","23,35",1,46.0,B,"Excadrill 38.536% ['ground', 'steel']",Arcanine 35.593% ['fire'],"Dragapult 33.117% ['dragon', 'ghost']"
1,059,Arcanine,[fire],90,110,80,100,80,95,"92,50","10,70",2,36.0,B,"Togekiss 45.687% ['fairy', 'flying']","Dragapult 30.663% ['dragon', 'ghost']","Tyranitar 28.847% ['rock', 'dark']"
2,887,Dragapult,"[dragon, ghost]",88,120,75,100,75,142,"100,00","24,35",3,31.0,B,"Togekiss 48.228% ['fairy', 'flying']","Excadrill 38.133% ['ground', 'steel']",Arcanine 34.788% ['fire']
3,530,Excadrill,"[ground, steel]",110,135,60,50,65,88,"84,67","29,95",4,30.0,B,"Togekiss 59.572% ['fairy', 'flying']","Tyranitar 47.122% ['rock', 'dark']","Dragapult 40.479% ['dragon', 'ghost']"
4,547,Whimsicott,"[grass, fairy]",60,67,85,77,75,116,"80,00","17,91",5,29.0,B-,"Duraludon 36.074% ['steel', 'dragon']","Togekiss 32.431% ['fairy', 'flying']",Arcanine 25.983% ['fire']


### Rename columns

In [32]:
Combined_DF.columns = ['National Dex #', 'Pokémon', 'Type', 'Base HP', 'Base Attack', 'Base Defense',
       'Base Special Attack', 'Base Special Defense', 'Base Speed', 'Base Stats Mean', 'Base Stats SD', 'VGC 2020 Usage Rank',
       'VGC 2020 Usage %', 'Competitive viability', 'Most frequent teammate',
       '2nd most frequent teammate', '3rd most frequent teammate']

In [34]:
Combined_DF["Base HP"] = Combined_DF["Base HP"].astype("float64")
Combined_DF["Base Attack"] = Combined_DF["Base Attack"].astype("float64")
Combined_DF["Base Defense"] = Combined_DF["Base Defense"].astype("float64")
Combined_DF["Base Special Attack"] = Combined_DF["Base Special Attack"].astype("float64")
Combined_DF["Base Special Defense"] = Combined_DF["Base Special Defense"].astype("float64")
Combined_DF["Base Speed"] = Combined_DF["Base Speed"].astype("float64")

Combined_DF["Base Stats Mean"] = Combined_DF["Base Stats Mean"].str.replace(',','.').astype(float)
Combined_DF["Base Stats SD"] = Combined_DF["Base Stats SD"].str.replace(',','.').astype(float)

### This is the resulting combined dataframe

In [35]:
Combined_DF.to_csv("/Users/Miguel/Documents/GitHub/Ironhack exercises/Modulo 1/project-web/Your-code/Output/Combined_DF_clean.csv")
Combined_Styler = Combined_DF.style.hide_index()

National Dex #,Pokémon,Type,Base HP,Base Attack,Base Defense,Base Special Attack,Base Special Defense,Base Speed,Base Stats Mean,Base Stats SD,VGC 2020 Usage Rank,VGC 2020 Usage %,Competitive viability,Most frequent teammate,2nd most frequent teammate,3rd most frequent teammate
468,Togekiss,"['fairy', 'flying']",85,50,95,120,115,80,90.83,23.35,1,46,B,"Excadrill 38.536% ['ground', 'steel']",Arcanine 35.593% ['fire'],"Dragapult 33.117% ['dragon', 'ghost']"
059,Arcanine,['fire'],90,110,80,100,80,95,92.5,10.7,2,36,B,"Togekiss 45.687% ['fairy', 'flying']","Dragapult 30.663% ['dragon', 'ghost']","Tyranitar 28.847% ['rock', 'dark']"
887,Dragapult,"['dragon', 'ghost']",88,120,75,100,75,142,100,24.35,3,31,B,"Togekiss 48.228% ['fairy', 'flying']","Excadrill 38.133% ['ground', 'steel']",Arcanine 34.788% ['fire']
530,Excadrill,"['ground', 'steel']",110,135,60,50,65,88,84.67,29.95,4,30,B,"Togekiss 59.572% ['fairy', 'flying']","Tyranitar 47.122% ['rock', 'dark']","Dragapult 40.479% ['dragon', 'ghost']"
547,Whimsicott,"['grass', 'fairy']",60,67,85,77,75,116,80,17.91,5,29,B-,"Duraludon 36.074% ['steel', 'dragon']","Togekiss 32.431% ['fairy', 'flying']",Arcanine 25.983% ['fire']
884,Duraludon,"['steel', 'dragon']",70,95,115,120,50,85,89.17,24.4,6,24,B-,"Whimsicott 43.844% ['grass', 'fairy']",Arcanine 39.353% ['fire'],"Grimmsnarl 31.799% ['dark', 'fairy']"
248,Tyranitar,"['rock', 'dark']",100,134,110,95,100,61,100,21.61,7,23,B,"Excadrill 61.114% ['ground', 'steel']","Togekiss 50.508% ['fairy', 'flying']",Arcanine 45.056% ['fire']
861,Grimmsnarl,"['dark', 'fairy']",95,120,65,95,75,60,85,20.62,8,20,B,Arcanine 42.733% ['fire'],"Duraludon 38.183% ['steel', 'dragon']","Togekiss 32.534% ['fairy', 'flying']"
130,Gyarados,"['water', 'flying']",95,125,79,60,100,81,90,20.22,9,17,B,"Togekiss 44.558% ['fairy', 'flying']","Excadrill 35.918% ['ground', 'steel']","Dragapult 31.131% ['dragon', 'ghost']"
423,Gastrodon,"['water', 'ground']",111,83,68,92,82,39,79.17,22.13,10,17,B,Arcanine 41.752% ['fire'],"Togekiss 40.461% ['fairy', 'flying']","Tyranitar 36.848% ['rock', 'dark']"


## Analyzing

### Look for outliers

In [36]:
#We will only consider lower outliers as the upper ones are basically VGC 2020 metagame's core
stats = Combined_DF.describe().transpose()
stats["IQR"] = stats["75%"] - stats ["25%"]
outliers = pd.DataFrame(columns=Combined_DF.columns)
for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = Combined_DF[(Combined_DF[col] < lower)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)

C:\Users\Miguel\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [37]:
stats

,count,mean,std,min,25%,50%,75%,max,IQR
Base HP,197.0,165.497462,874.418253,1.00,60.00,72.00,90.00,9026.00,30.00
Base Attack,197.0,361.949239,1720.418443,10.00,65.00,90.00,110.00,13033.00,45.00
Base Defense,197.0,124.705584,464.130210,40.00,65.00,80.00,100.00,6527.00,35.00
Base Special Attack,197.0,95.796954,130.849810,10.00,57.00,79.00,95.00,1106.00,38.00
Base Special Defense,197.0,125.751269,567.380567,20.00,67.00,80.00,95.00,8030.00,28.00
Base Speed,197.0,73.477157,79.507577,5.00,45.00,70.00,88.00,1106.00,43.00
Base Stats Mean,197.0,79.980508,9.723294,39.33,77.50,81.67,85.00,100.00,7.50
Base Stats SD,197.0,23.774416,11.239695,0.00,16.58,23.21,28.72,103.22,12.14
VGC 2020 Usage %,196.0,2.770408,6.686637,0.00,0.00,0.00,1.00,46.00,1.00


### We clean and manipulate this new "outliers" dataframe 

In [38]:
outliers = outliers.drop(['2nd most frequent teammate', '3rd most frequent teammate',
        'Competitive viability', 'Most frequent teammate',
       'National Dex #', 'VGC 2020 Usage Rank'], axis=1)

In [39]:
column_order = ['Outlier', 'Pokémon', 'Type', 'Base Attack', 'Base Defense', 'Base HP', 'Base Special Attack',
       'Base Special Defense', 'Base Speed', 'Base Stats Mean',
       'Base Stats SD', 'VGC 2020 Usage %']
outliers = outliers[column_order]

### Lower outliers = Pokémon that are very rarely used in VGC and we should be careful with

In [40]:
outliers.to_csv("/Users/Miguel/Documents/GitHub/Ironhack exercises/Modulo 1/project-web/Your-code/Output/outliers_clean.csv")
outliers.head()

,Outlier,Pokémon,Type,Base Attack,Base Defense,Base HP,Base Special Attack,Base Special Defense,Base Speed,Base Stats Mean,Base Stats SD,VGC 2020 Usage %
175,Base HP,Shedinja,"[bug, ghost]",90.0,45.0,1.0,30.0,30.0,40.0,39.33,26.59,0.0
101,Base Special Defense,Stonjourner,[rock],125.0,135.0,100.0,20.0,20.0,70.0,78.33,46.07,0.0
33,Base Stats Mean,Butterfree,"[bug, flying]",45.0,50.0,60.0,901.0,80.0,70.0,65.83,15.92,5.0
45,Base Stats Mean,Sableye,"[dark, ghost]",75.0,75.0,50.0,65.0,65.0,50.0,63.33,10.27,2.0
52,Base Stats Mean,Pikachu,[electric],55.0,404.0,35.0,50.0,505.0,90.0,53.33,17.72,1.0
133,Base Stats Mean,Ditto,[normal],48.0,48.0,48.0,48.0,48.0,48.0,48.00,0.00,0.0
143,Base Stats Mean,Gothorita,[psychic],45.0,70.0,60.0,75.0,85.0,55.0,65.00,13.23,0.0
159,Base Stats Mean,Dottler,"[bug, psychic]",35.0,80.0,50.0,50.0,90.0,30.0,55.83,22.06,0.0
161,Base Stats Mean,Delibird,"[ice, flying]",55.0,45.0,45.0,65.0,45.0,75.0,55.00,11.55,0.0
167,Base Stats Mean,Clefairy,[fairy],45.0,48.0,70.0,60.0,65.0,35.0,53.83,12.18,0.0


### Correlation between base stats mean and usage

In [41]:
print(Combined_DF["VGC 2020 Usage %"].corr(Combined_DF["Base Stats Mean"]))
print(outliers["VGC 2020 Usage %"].corr(outliers["Base Stats Mean"]))

0.29521109137099183
0.31303949037067785


It seems that "lower" outliers are more depandant on their base stats mean compared to the population overall but there's little difference so wou shoul try to focus on other variables. I personally think it could be their battle roles, but this information was not in any of the datasets I worked with or that were at least discarded candidates. 